In [105]:
import pandas as pd

df = pd.read_csv('./train_combined_Species.csv')
df.drop(df.columns[0], axis=1, inplace=True)
df = df.fillna(0).drop_duplicates()

def drop_almost_zero(df, percentage):
    column_cut_off = int(percentage/100*len(df))
    b = (df == 0).sum(axis='rows')
    df = df[ b[ b <= column_cut_off].index.values ]
    return df
#df = drop_almost_zero(df,90)

inputvalues = df.drop(['label'], axis=1)
outputvalues = df['label']
X, y = inputvalues.values, outputvalues.values
X = X.astype('float32')

In [106]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import pickle as pk

scaler = MinMaxScaler().fit(X)
pk.dump(scaler, open("scaler.pkl","wb"))
X = scaler.transform(X)

pca = PCA(n_components=90).fit(X)
pk.dump(pca, open("pca.pkl","wb"))
X = pca.transform(X)

In [107]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

sampler = RandomOverSampler(sampling_strategy='all')
X_train, y_train = sampler.fit_resample(X_train, y_train)

In [108]:
import joblib
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn import svm

clf = svm.SVC(kernel='linear', C=1)
clf.fit(X_train, y_train)
joblib.dump(clf, 'Species_HuyenHanhFlorianElaMohammedDominik')
y_pred = clf.predict(X_test)
print("Classification Report: \n {}\n".format(metrics.classification_report(y_test, y_pred)))
print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, y_pred)))

Classification Report: 
               precision    recall  f1-score   support

           0       0.48      0.58      0.52        85
           1       0.53      0.26      0.35        93
           2       0.07      0.17      0.10        12
           3       0.12      0.25      0.17         8
           4       0.09      0.25      0.13         4

    accuracy                           0.39       202
   macro avg       0.26      0.30      0.25       202
weighted avg       0.46      0.39      0.39       202


Confusion Matrix: 
 [[49 18 10  4  4]
 [44 24 13  8  4]
 [ 6  1  2  2  1]
 [ 3  0  2  2  1]
 [ 1  2  0  0  1]]

